In [1]:
!pip install -r requirements.txt

In [2]:
import json
import sys
import os
import re
sys.path.append("src")

In [3]:
%load_ext autoreload
%autoreload 2

In [31]:
%reload_ext autoreload

In [4]:
from wiki_extract import Extracter
from wiki_gen import WikiGen
from openai_utils import LlmCompleter
from openai_utils import AsyncList
from wiki_metrics import WikiEval

resource module not available on Windows


In [4]:
def get_article(name, retrieve_sources=False):
    print('Article name: ', name)
    page = Extracter(name)
    page.get_references()
    page.get_outline()
    if retrieve_sources:
        page.fast_extracter()
        page.get_filtered_outline()
    return page

In [11]:
article_names = [
    'Python',
    'Летние Олимпийские игры 2024',
    'Квантовый компьютер',
    'Присоединение Крыма к Российской Федерации',
    'Сколково (инновационный центр)',
    'Tomb Raider (игра, 2013)',
    'Чёрная дыра',
    'Экономика США',
    'Искусственный интеллект',
    'COVID-19',
    'Применение искусственного интеллекта',
    'РИА Новости',
    'Uncharted 4: A Thief’s End',
    'Экономика КНР',
    'Экономика Германии',
    'Иннополис',
    'Летние Олимпийские игры 2020',
    'Солнечная система',
    'C++',
    'Си (язык программирования)',
    'Сбербанк России',
    'Чёрная смерть',
    'Яндекс',
    'Google (компания)',
    'Большой адронный коллайдер',
    'Геморрагическая лихорадка Эбола',
    'Вирус иммунодефицита человека',
    'Dota 2',
    'TikTok',
    'Марс',
    'YouTube'
]

In [10]:
page = get_article('YouTube', True)

Article name:  YouTube


Calculating reference positions: 100%|██████████| 18/18 [00:00<?, ?it/s]


In [25]:
for name in article_names:
    page = get_article(name, True)

Article name:  Python


Calculating reference positions: 100%|██████████| 54/54 [00:00<00:00, 54042.57it/s]

Article name:  Летние Олимпийские игры 2024



Calculating reference positions: 100%|██████████| 32/32 [00:00<00:00, 32078.81it/s]

Article name:  Квантовый компьютер


...


Calculating reference positions: 100%|██████████| 30/30 [00:00<?, ?it/s]

Article name:  Google (компания)



Calculating reference positions: 100%|██████████| 26/26 [00:00<00:00, 26001.88it/s]

Article name:  Большой адронный коллайдер



Calculating reference positions: 100%|██████████| 32/32 [00:00<?, ?it/s]


In [7]:

client = LlmCompleter(api_address=URL, api_key=KEY, model_name='llama3-70b')
wiki_writer = WikiGen(client)
annotations = AsyncList()

pages = [get_article(name) for name in article_names]

for page in pages:
    annotations.append(wiki_writer.get_ref_subqueries(page))
    annotations.append(wiki_writer.get_ref_subqueries(page, 1))

await annotations.complete_couroutines(batch_size=20)
annotations = await annotations.to_list()

Article name:  Python


Creating outline: 100%|██████████| 208/208 [00:00<00:00, 41597.06it/s]


Article name:  Летние Олимпийские игры 2024


Creating outline: 100%|██████████| 109/109 [00:00<00:00, 71940.07it/s]


Article name:  Квантовый компьютер


...

Creating outline: 100%|██████████| 105/105 [00:00<00:00, 52341.56it/s]


Article name:  Google (компания)


Creating outline: 100%|██████████| 142/142 [00:00<00:00, 47302.93it/s]


Article name:  Большой адронный коллайдер


Creating outline: 100%|██████████| 88/88 [00:00<00:00, 44668.86it/s]


In [10]:
dir_name = os.path.join("Generation", "Subqueries_ref", "Annotations")
for i in range(0, len(annotations), 2):
    name = re.sub(r'[<>:"/\\|?*]', '', annotations[i][0])
    file_name = os.path.join(dir_name, name + '.txt')
    with open(file_name, 'w', encoding='utf-8') as f:
        f.write(annotations[i][1] + "\n")
        f.write(annotations[i + 1][1])

In [46]:
%reload_ext autoreload

In [ ]:
'''
for path in leaf_paths:
    answer.append(wiki_writer.get_ref_subqueries(page.name, path))

await answer.complete_couroutines(batch_size=25)
answer = await answer.to_list()


for i, json_str in enumerate(answer):
    if json_str:
        try:
            if json_str.startswith('`'):
                json_str = json_str[7:-3]
            data = json.loads(json_str)
            directory = os.path.join("Generation", "Subqueries_ref", page.name)
            os.makedirs(directory, exist_ok=True)
            filename = page.name.replace(" ", "_") + '_' + str(i) + '.json'
            file_path = os.path.join(directory, filename)
            with open(file_path, "w", encoding="utf-8") as f:
                json.dump(data, f, ensure_ascii=False, indent=2)
        except:
            print(f"Пропущен некорректный JSON: {json_str}...")
            continue
'''

In [22]:
texts = []
for subdir, _, files in os.walk(r'Generation\Subqueries_ref\Annotations'):
    for file in files:
        if file.endswith(".txt"):
            file_path = os.path.join(subdir, file)
            with open(file_path, "r", encoding="utf-8") as f:
                text = f.read()
                texts.append((text, file_path))

In [23]:
print(texts[0][1].split('\\')[-1].split('.')[0])

C++


In [15]:
import os

base_dir = os.path.join('Articles', 'Sources')

folders = [f for f in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, f))]
txt_counts = []

for folder in folders:
    folder_path = os.path.join(base_dir, folder)
    txt_files = [file for file in os.listdir(folder_path) if file.lower().endswith('.txt')]
    txt_counts.append(len(txt_files))

for folder, count in zip(folders, txt_counts):
    print(f'{folder}: {count} .txt файлов')


C++: 12 .txt файлов
COVID-19: 90 .txt файлов
Google (компания): 56 .txt файлов
Python: 76 .txt файлов
Tomb Raider (игра, 2013): 68 .txt файлов
Uncharted 4 A Thief’s End: 27 .txt файлов
Большой адронный коллайдер: 24 .txt файлов
Броненосный крейсер: 4 .txt файлов
Иннополис: 45 .txt файлов
Искусственный интеллект: 37 .txt файлов
Квантовый компьютер: 27 .txt файлов
Летние Олимпийские игры 2020: 10 .txt файлов
Летние Олимпийские игры 2024: 39 .txt файлов
Применение искусственного интеллекта: 82 .txt файлов
Присоединение Крыма к Российской Федерации: 20 .txt файлов
РИА Новости: 35 .txt файлов
Сбербанк России: 103 .txt файлов
Си (язык программирования): 62 .txt файлов
Сколково (инновационный центр): 156 .txt файлов
Солнечная система: 32 .txt файлов
Чёрная дыра: 35 .txt файлов
Чёрная смерть: 18 .txt файлов
Экономика Германии: 5 .txt файлов
Экономика КНР: 65 .txt файлов
Экономика США: 25 .txt файлов
Яндекс: 50 .txt файлов


In [19]:


snippets = wiki_evaluater.load_corpus()


import os

base_dir = os.path.join('Articles', 'Sources')

folders = [f for f in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, f))]

topR = []

for folder in folders:
    snip = [1 if snippet['doc_id'].split("\\")[-2] == folder else 0 for snippet in snippets]
    topR.append((folder, sum(snip)))

Split strings:   0%|          | 0/2001 [00:00<?, ?it/s]

Stem Tokens:   0%|          | 0/2001 [00:00<?, ?it/s]

{'Articles\\Sources\\Dota 2\\source_20.txt', 'Articles\\Sources\\Python\\source_10.txt', 'Articles\\Sources\\Python\\source_23.txt', 'Articles\\Sources\\Python\\source_26.txt', 'Articles\\Sources\\Иннополис\\source_16.txt', 'Articles\\Sources\\Сколково (инновационный центр)\\source_40.txt', 'Articles\\Sources\\Геморрагическая лихорадка Эбола\\source_23.txt', 'Articles\\Sources\\Применение искусственного интеллекта\\source_82.txt', 'Articles\\Sources\\Экономика США\\source_4.txt', 'Articles\\Sources\\Большой адронный коллайдер\\source_24.txt', 'Articles\\Sources\\COVID-19\\source_5.txt', 'Articles\\Sources\\TikTok\\source_33.txt', 'Articles\\Sources\\Uncharted 4 A Thief’s End\\source_19.txt', 'Articles\\Sources\\Сколково (инновационный центр)\\source_10.txt', 'Articles\\Sources\\Яндекс\\source_36.txt', 'Articles\\Sources\\Сколково (инновационный центр)\\source_98.txt', 'Articles\\Sources\\Google (компания)\\source_30.txt', 'Articles\\Sources\\TikTok\\source_32.txt', 'Articles\\Sources\\

Saving the dataset (0/1 shards):   0%|          | 0/2001 [00:00<?, ? examples/s]

BM25S Count Tokens:   0%|          | 0/2001 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/2001 [00:00<?, ?it/s]

Finding newlines for mmindex:   0%|          | 0.00/9.30M [00:00<?, ?B/s]

In [20]:
print(topR)

[('C++', 15), ('COVID-19', 135), ('Dota 2', 42), ('Google (компания)', 63), ('Portal 2', 40), ('Python', 177), ('TikTok', 77), ('Tomb Raider (игра, 2013)', 76), ('Uncharted 4 A Thief’s End', 33), ('YouTube', 31), ('Большой адронный коллайдер', 24), ('Вирус иммунодефицита человека', 58), ('Геморрагическая лихорадка Эбола', 38), ('Иннополис', 51), ('Искусственный интеллект', 59), ('Квантовый компьютер', 29), ('Летние Олимпийские игры 2020', 16), ('Летние Олимпийские игры 2024', 40), ('Марс', 66), ('Применение искусственного интеллекта', 108), ('Присоединение Крыма к Российской Федерации', 30), ('РИА Новости', 37), ('Сбербанк России', 104), ('Си (язык программирования)', 134), ('Сколково (инновационный центр)', 177), ('Солнечная система', 61), ('Чёрная дыра', 61), ('Чёрная смерть', 58), ('Экономика Германии', 5), ('Экономика КНР', 78), ('Экономика США', 25), ('Яндекс', 53)]


In [33]:
client = LlmCompleter(api_address=URL, api_key=KEY, model_name='llama3-70b')
wiki_writer = WikiGen(client)
wiki_evaluater = WikiEval(wiki_writer, True)

for (annotation, path), (_, topK) in zip(texts, topR):
    ndcg, pr_r_score = await wiki_evaluater.rank_one_query(annotation, 3*topK, path.split('\\')[-1].split('.')[0])
    print(path.split('\\')[-1].split('.')[0])
    print("Ndcg: ", round(ndcg, 2))
    print("R-precision: ", round(pr_r_score, 2))

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

Stem Tokens:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

C++
Ndcg:  0.98
R-precision:  0.86


Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

...

Экономика США
Ndcg:  0.93
R-precision:  0.79


Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

Stem Tokens:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Яндекс
Ndcg:  0.98
R-precision:  0.9


In [32]:
%reload_ext autoreload

In [44]:
wiki_evaluater = WikiEval(wiki_writer)
pr = 32
ndcg, pr_r_score = await wiki_evaluater.rank_one_query(texts[0][0], 30, texts[0][1].split('\\')[-1].split('.')[0], pr)
print(texts[17][1].split('\\')[-1].split('.')[0])
print("Ndcg: ", ndcg)
print("R-precision: ", pr_r_score)

Split strings:   0%|          | 0/1656 [00:00<?, ?it/s]

Stem Tokens:   0%|          | 0/1656 [00:00<?, ?it/s]

Saving the dataset (0/1 shards):   0%|          | 0/1656 [00:00<?, ? examples/s]

BM25S Count Tokens:   0%|          | 0/1656 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/1656 [00:00<?, ?it/s]

Finding newlines for mmindex:   0%|          | 0.00/8.12M [00:00<?, ?B/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

Stem Tokens:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Си (язык программирования)
Ndcg:  1.0
R-precision:  0.15625
